In [35]:
import datetime as dt
import meteomatics.api as api
from datetime import datetime
import pandas as pd
from config import USERNAM, PASSWORD
from constants import cities_coordinates

# Usuario
username = USERNAM
password = PASSWORD


# Coordenadas de las ubicaciones deseadas
coordinates = [(city, coord[0], coord[1]) for city, coord in cities_coordinates.items()]

# Parámetros climáticos a solicitar
parameters = ['t_2m:C', 'precip_1h:mm', 'wind_speed_10m:ms']
model = 'mix'

# Definir el rango de fechas de consulta (próximos 2 días)
startdate = dt.datetime.utcnow().replace(minute=0, second=0, microsecond=0)
enddate = startdate + dt.timedelta(days=2)
interval = dt.timedelta(hours=6)

# Realizar la consulta a la API para cada ubicación
dfs = []
for city, lat, lon in coordinates:
    df = api.query_time_series([(lat, lon)], startdate, enddate, interval, parameters, username, password, model=model)
    df['Ciudad'] = city  # Agregar la columna de Ciudad
    dfs.append(df)

# Combinar los resultados de cada ubicación en un solo DataFrame
df_final = pd.concat(dfs)

# Agregar la columna de Fecha_Actualizacion
fecha_actualizacion = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_final['Fecha_Actualizacion'] = fecha_actualizacion

# Reordenar las columnas
df_final = df_final[['Ciudad', 't_2m:C', 'precip_1h:mm', 'wind_speed_10m:ms', 'Fecha_Actualizacion']]
df_final.columns = ['Ciudad', 'Temperatura C', 'Precipitacion mm/h', 'Velocidad de viento m/s', 'Fecha_Actualizacion']
df_final.reset_index(inplace=True)

In [36]:
import psycopg2
from config import dbname, user, clave, host, port2

#cedenciales
conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=clave,
    host=host,
    port=port2
)

# Crear cursor
cur = conn.cursor()

# Insertar datos
sql = "INSERT INTO clima VALUES (%s, %s, %s, %s, %s, %s, %s, %s);"


for index, row in df_final.iterrows():
    cur.execute(sql, ( row['lat'],row['lon'] ,row['validdate'], row['Ciudad'], row['Temperatura C'], row['Precipitacion mm/h'], 
                row['Velocidad de viento m/s'], row['Fecha_Actualizacion']))

# confirmar cambios
conn.commit()

# Close cursor and connection
cur.close()
conn.close()